In [1]:
import math
import random

### Helper Functions 

In [2]:
def scalarVectorProduct(scalar, vector):
    """
    This function multiplies a scalar with a vector.

    Parameters
    ----------
    scalar : float
        Scalar.
    vector : list
        List of numbers.
    """

    output = []
    for i in range(len(vector)):
        output.append(scalar * vector[i])

    return output

In [3]:
def addTwoVectors(vector1, vector2):
    """
    This function adds two vectors.

    Parameters
    ----------
    vector1 : list
        List of numbers.
    vector2 : list
        List of numbers.
    """

    output = []
    for i in range(len(vector1)):
        output.append(vector1[i] + vector2[i])

    return output

In [4]:
def generateRandomMatrix(size, density, elements_range=(-5, 5)):
    """
    Generates a random matrix given the density of sparseness.
    
    Parameters
    ----------
    size : tuple 
        The size of the matrix.
    elements_range : tuple
        The range of the elements in the matrix.
    density : float
        The density of sparseness of the matrix.
        
    Returns
    -------
    matrix : list of lists
        The random sparse matrix.
    """
    matrix = []
    for i in range(size[0]):
        row = []
        for j in range(size[1]):
            # random.random() generates a number between 0 and 1 uniformly
            # if random.random() < density, then the element is non-zero otherwise it is zero
            if random.random() < density: 
                row.append(random.randint(elements_range[0], elements_range[1]))
            else:
                row.append(0)
        matrix.append(row)
    return matrix

### Main Code

In [5]:
class GaussianElimination(object):
    """
    A class to solve a system of linear equations using Gaussian Elimination
    """
    def __init__(self):
        """
        Constructor

        Initializes/Parameters
        ----------------------
        A : list of lists
            A matrix of size m x n
        b : list
            b vector of size m
        x : list
            x vector of size n
        m : int
            number of rows in A
        n : int
            number of columns in A
        """

        self.A = []
        self.b = []
        self.x = []
        self.m = 0
        self.n = 0

    def fit(self, A, b):
        """
        Solves the system of linear equations

        Parameters
        ----------
        A : list of lists
            A matrix of size m x n
        b : list
            b vector of size m
        """

        # Initializing the parameters
        self.A = A.copy()
        self.b = b.copy()
        self.m = len(self.A)
        self.n = len(self.A[0])

        self.printQuestion()

        # Performing forward elimination
        self.forwardElimination()
        solution_type = self.checkSolutionType()
        print("The system of linear equations have ", solution_type)

        if solution_type == "Unique solution":
            self.x = [0]*self.n
            self.backSubstitution()
            
            print("x = ", self.x)
        
        
    def forwardElimination(self):
        """
        Performs forward elimination on the matrix A
        """
        
        pivot = 0    # column location for the pivot 
        row_num = 0 
        while pivot < self.n and row_num < self.m:
            # flag to check if we could find a non-zero element in the column below kth row (including kth row)
            flag = self.swapRows(pivot, row_num)

            if flag == 0: # if flag = 0 means that the column is all zeros below kth row (no swapping)
                pivot += 1 
            else:
                for i in range(row_num+1, self.m):                
                    factor = self.A[i][pivot]/self.A[row_num][pivot]
                    self.A[i] = addTwoVectors(self.A[i], scalarVectorProduct(-factor, self.A[row_num])) # R_i = R_i - factor*R_(row_num)
                    self.b[i] = self.b[i] - factor*self.b[row_num]
                
                pivot += 1
                row_num += 1
    
    def backSubstitution(self):
        """
        Performs back substitution on the matrix A
        """

        for i in range(self.m-1, -1, -1):
            # Solving for x_i using already solved x_j's where j > i
            summation = 0
            for j in range(i+1, self.n): 
                summation += self.A[i][j]*self.x[j]
            
            self.x[i] = (self.b[i] - summation)/self.A[i][i]
    
    def checkSolutionType(self):
        """
        Tells whether the solution is unique, infinite or no solution after forward elimination

        Returns
        -------
        str
            "Unique solution" if the solution is unique
            "Infinite solution" if the solution is infinite
            "No solution" if the solution is no solution
        """

        # Unique Solution if the last row of A is not all zeros
        if not self.checkAllZeroRow(self.m-1):
            return "Unique solution"

        # If there exists a row with all zeros and b[i] != 0 then No Solution otherwise Infinite Solution
        # We start from the last row and check if it is all zeros and b[i] != 0. 
        # We check this until we find a row which is not all zeros or we reach the first row.

        for i in range(self.m - 1, -1, -1):
            if self.checkAllZeroRow(i):
                if self.b[i] != 0: 
                    return "No solution" 
            else: 
                # found a row which is not all zeros
                break
        
        # If all the rows with all zeros have b[i] = 0 then Infinite Solution
        return "Infinite solution"
    
    def swapRows(self, pivot, k):
        """
        Swaps rows if the kth element in the kth column is zero

        Parameters
        ----------
        pivot : int
            Column number
        k : int
            Row number
        
        Returns
        -------
        bool
            True if A[k][k] != 0 otherwise False
        """

        if self.A[k][pivot] != 0:
            return True
        else:
            for j in range(k+1, self.m):
                if self.A[j][pivot] != 0:
                    self.A[k], self.A[j] = self.A[j], self.A[k]
                    self.b[k], self.b[j] = self.b[j], self.b[k]
                    return True
            return False
    
    def checkAllZeroRow(self, k):
        """
        Checks if a row is all zeros

        Parameters
        ----------
        k : int
            Row number
        
        Returns
        -------
        bool
            True if row is all zeros otherwise False
        """

        for j in range(self.n):
            if self.A[k][j] != 0:
                return False
        return True

    def printQuestion(self):
        print("For A = ")
        for i in range(self.m):
            print(self.A[i])
        print("and b = ", self.b)
        print()

In [6]:
solver = GaussianElimination()

In [7]:
# Infinite solutions Example
A = [[1, 2, 3], [4, 5, 6], [7, 8, 9]] # A matrix with linearly dependent columns
b = [1, 2, 3] # b vector is in the column space of A

solver.fit(A, b)

For A = 
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
and b =  [1, 2, 3]

The system of linear equations have  Infinite solution


In [8]:
# No solution Example
A = [[4, 1, 0, 0, -2], [0.0, 5.0, 0.0, 3.0, -2.0], [0.0, 0.0, 0.0, 0.0, -5.0], [0.0, 0.0, 0.0, -3.4, 6.6], [0.0, 0.0, 0.0, 0.0, 5.0]]
b = [-1, -6.0, 3.0, 7.8, 1.0]

solver.fit(A, b)

For A = 
[4, 1, 0, 0, -2]
[0.0, 5.0, 0.0, 3.0, -2.0]
[0.0, 0.0, 0.0, 0.0, -5.0]
[0.0, 0.0, 0.0, -3.4, 6.6]
[0.0, 0.0, 0.0, 0.0, 5.0]
and b =  [-1, -6.0, 3.0, 7.8, 1.0]

The system of linear equations have  No solution


In [9]:
# Unique solution Example
A = [[1, 2, 3], [4, 5, 6], [3, 2, 7]] # A matrix with linearly independent columns
b = [1, 2, 3] # b vector is in the column space of A

solver.fit(A, b)

For A = 
[1, 2, 3]
[4, 5, 6]
[3, 2, 7]
and b =  [1, 2, 3]

The system of linear equations have  Unique solution
x =  [0.11111111111111116, -0.22222222222222218, 0.4444444444444444]


In [10]:
# Checking the solution for the Unique solution Example (Using numpy to check)
import numpy as np
A = np.array(A)
b = np.array(b)
x = np.linalg.solve(A, b)
print("x = ", x)

x =  [ 0.11111111 -0.22222222  0.44444444]


In [11]:
# Random Matrix
A = generateRandomMatrix((5, 5), 0.5, (-5, 5))
b = [random.randint(-5, 5) for i in range(5)]
solver.fit(A, b)

For A = 
[0, 0, 3, 0, -5]
[0, 0, -5, 0, 0]
[1, 0, -2, 2, 0]
[5, -2, 2, 0, 0]
[-3, 0, 0, 0, 4]
and b =  [0, 2, 0, 2, -5]

The system of linear equations have  Unique solution
x =  [1.3466666666666667, 1.9666666666666663, -0.39999999999999997, -1.0733333333333333, -0.24]


In [12]:
# Zero Matrix Infinite Solution
A = [[0,0],[0,0]]
b = [0,0]

solver.fit(A, b)

For A = 
[0, 0]
[0, 0]
and b =  [0, 0]

The system of linear equations have  Infinite solution


In [13]:
# Zero Matrix No Solution
A = [[0,0],[0,0]]
b = [1,0]

solver.fit(A, b)

For A = 
[0, 0]
[0, 0]
and b =  [1, 0]

The system of linear equations have  No solution
